# Análise dos produtos comercializados

## Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = APIKEYHERE
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


## Inicializar a API e transferir os códigos usados

In [3]:
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)

## China, categorias de produtos mais importantes nas trocas com os PLP

Obtem os totais agregados com nível 2 de código HS e lista os primeiros.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx


### TODO

- [ ] Permitir escolher os reporter e partners
- [ ] compatibilizar com o formato de report da AICEP https://myaicep.portugalexporta.com/mercados-internacionais/cn/china?setorProduto=-1
- [ ] há problemas de duplicação de linhas nas listagem de detalhes de commodities, provavlmente por custom code, ou mot code.
  

In [4]:
import comtrade
import pandas as pd


rank_filter = 5  # número de importações mais relevantes

year_start = 2020
year_end = 2023
years = comtrade.year_range(year_start,year_end)
reporterCode = comtrade.m49_china
partnerCode = comtrade.m49_plp_list
flowCode = 'M,X'
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
            'flowCode','primaryValue']



pco_top5_sorted = comtrade.top_commodities(reporterCode, partnerCode,  years, flowCode,rank_filter, pco_cols)

# save to Excel
# Prepare file name for Excel output
filename_note=f"{year_start}-{year_end}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./reports/china_plp_top5_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)

# Save data
sheet_name = "comtrade"
pco_top5_sorted.to_excel(excel_file, sheet_name=sheet_name, index=True, startrow=2)
# Adjust column width
comtrade.excel_col_autowidth(pco_top5_sorted, excel_file,sheet_name)

# format column primaryValue as currency
comtrade.excel_format_currency(pco_top5_sorted, excel_file,sheet_name, columns=['primaryValue'],width=20)
    
# Write title in first row TBD
excel_file_title = f"Trocas Comerciais China-PLP, top {rank_filter} produtos (M=Importações, X=Exportações), {years}, valores USD"
excel_file.sheets[sheet_name].write(0, 0, excel_file_title)
print("Guardado em:",excel_file_name)

excel_file.close()
# show
pd.options.display.max_colwidth=100
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 100
pco_top5_sorted.head(50)

Guardado em: ./reports/china_plp_top5_2020-2023_M_X.xlsx


cmdCode  \
reporterDesc partnerDesc       flowCode refYear rank           
China        Angola            M        2020    1         27   
                                                2         25   
                                                3         71   
                                                4         74   
                                                5         26   
                               X        2020    1         85   
                                                2         84   
                                                3         64   
                                                4         72   
                                                5         39   
             Brazil            M        2020    1         26   
                                                2         12   
                                                3         27   
                                                4         02   
                                                5         47   
                               X        2020    1         85   
                                                2         84   
                                                3         29   
                                                4         90   
                                                5         87   
             Cape Verde        M        2020    1         74   
                                                2         84   
                                                3         85   
                                                4         99   
                                                5         63   
                               X        2020    1         85   
                                                2         16   
                                                3         84   
                                                4         94   
                                                5         73   
             Equatorial Guinea M        2020    1         27   
                                                2         44   
                                                3         99   
                                                4         63   
                               X        2020    1         73   
                                                2         39   
                                                3         84   
                                                4         69   
                                                5         85   
             Mozambique        M        2020    1         26   
                                                2         44   
                                                3         12   
                                                4         03   
                                                5         27   
                               X        2020    1         39   
                                                2         64   
                                                3         61   
                                                4         63   
                                                5         27   

                                                                                                                                                  cmdDesc  \
reporterDesc partnerDesc       flowCode refYear rank                                                                                                        
China        Angola            M        2020    1     Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral w...   
                                                2                                     Salt; sulphur; earths, stone; plastering materials, lime and cement   
                                                3     Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with prec...   
   

## Análise de variações nos produtos mais importantes

A célula seguinte produz uma lista das variações mais importantes na evolução das 
trocas comerciais entre a China e os Países de Língua Portuguesa

Por cada combinação de países e de fluxo (M=importações pela China, X=exportações da China)
por exemplo:

        China Brazil M

        1993   72,26
                -> 26   141,782,165.00 Ores, slag and ash
                -> 72   540,603,898.00 Iron and steel
        1994 ! 15,72
                -> 15   463,690,083.00 Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes
        1995 ! 15,26
        1996 ! 15,23
                -> 23   270,712,928.00 Food industries, residues and wastes thereof; prepared animal fodder
        1997 ≈ 23,15
        1998 ! 26,23
        1999 ! 26,12
                -> 12   171,963,034.00 Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder
        2000 ≈ 12,26
        2001 ≈ 26,12
        2002 ≈ 12,26
        2003 =
        2004 ≈ 26,12

Cada ano é seguido de um sinal que indica a mudança ocorrida em relação ao ano anterior:
* ! : mudança num dos produtos principais
* = : mesmos produtos do ano anterior, pela mesma ordem de importância
* ≈ : mesmos produtos do ano anterior, mas importância diferente


Cada vez que um novo produto aparece na série é fornecida a descrição do código respectivo e o valor envolvido.

Esta listagem pode ser produzida tendo em conta um número variáveis de produtos principais
normalmente os dois principais (ver variável `consider_first`)

In [5]:
consider_first = 2
for reporter in pco_top5_sorted.index.get_level_values(0).unique():
    for partner in pco_top5_sorted.loc[(reporter,)].index.get_level_values(0).unique(): 
        for flow in pco_top5_sorted.loc[((reporter,partner,))].index.get_level_values(0).unique():
            print(reporter,partner, flow)
            previous_pattern = []
            all_commodities = set() # all commodities for this partner for this flow
            all_commodities_desc = set() # all commodities for this partner for this flow descriptions
            previous_years_values = dict()
            for year in pco_top5_sorted.loc[(reporter,partner,flow,)].index.get_level_values(0).unique():
                print(year, end=' ')
                top5_codes = pco_top5_sorted.loc[(reporter,partner,flow,year)]['cmdCode'].astype(str).values.flatten().tolist()[0:consider_first]
                top5_codes_desc = pco_top5_sorted.loc[(reporter,partner,flow,year)]['cmdDesc'].astype(str).values.flatten().tolist()[0:consider_first]
                top5_primary_value = pco_top5_sorted.loc[(reporter,partner,flow,year)]['primaryValue'].values.flatten().tolist()[0:consider_first]
                top5_table = {code: {"desc":desc, "value":value, "year": year} for code,desc,value in zip(top5_codes,top5_codes_desc,top5_primary_value)}
                codes_desc = {f"{code} {desc}" for code,desc in zip(top5_codes,top5_codes_desc)}
                # make set of tuples top5_codes and top5_codes_desc
                new_commodities = set(top5_codes).difference(all_commodities)
                all_commodities = all_commodities.union(set(top5_codes))
                all_commodities_desc = all_commodities_desc.union(codes_desc)
                pattern = ','.join(top5_codes)
                if len(previous_pattern) == 0:
                    previous_pattern = top5_codes
                    # update previous_years_values with top5_table
                    previous_years_values.update(top5_table)
                    print(' ', pattern, end=' | ')
                    for cmd in top5_codes:
                        print(f"{top5_table[cmd]['value']:,.2f} ({top5_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                    for cmd in new_commodities:
                        print( " "*13+"  ->", cmd,top5_table[cmd]['desc'])
                elif set(top5_codes) == set(previous_pattern):
                    if top5_codes == previous_pattern:
                        print('=', pattern, end=' | ')
                    else:
                        print('≈', pattern, end=' | ')
                    for cmd in top5_codes:
                        print(f"{top5_table[cmd]['value']:,.2f} ({top5_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                else:
                    print('!', pattern, end=' | ')
                    for cmd in top5_codes:
                        print(f"{top5_table[cmd]['value']:,.2f} ", end=' ')
                        if cmd in previous_years_values.keys():
                            print(f"({top5_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                        else:
                            print(f"(new:{cmd})", end=' | ')
                    print()
                    if len(new_commodities) > 0:
                        for cmd in new_commodities:
                            print( " "*13+"  ->", cmd, top5_table[cmd]['desc'])
                       
                previous_pattern = top5_codes
                previous_years_values.update(top5_table)
            for cmd in sorted(all_commodities_desc):
                print( "    ",cmd)
            print(80*"-")



China Angola M
2020   27,25 | 14,686,950,196.00 (100%) | 29,428,961.00 (100%) | 
               -> 25 Salt; sulphur; earths, stone; plastering materials, lime and cement
               -> 27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
     25 Salt; sulphur; earths, stone; plastering materials, lime and cement
     27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
--------------------------------------------------------------------------------
China Angola X
2020   85,84 | 203,996,443.00 (100%) | 137,859,846.00 (100%) | 
               -> 85 Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles
               -> 84 Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof
     84 Machinery and mechanical appliances, boilers, nuclear r

## Detalhe do tipo de produtos comercializados entre os PLP e a China

Em dois passos: 
* obtém as categorias de nível 2 mais importantes de cada país
* pesquisa todos as subcategorias de cada um das categorias

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [6]:

import ipywidgets as widgets
from IPython.display import display
import comtrade

rank_filter = 5  # número de importações mais relevantes
years = "2021"
partnerCode = comtrade.m49_angola # 
flowCode="M"

# select year with widget from list range(2000,2022)

years = widgets.SelectMultiple(
    options=range(2000,2022),
    value=[2021],
    description='Ano:',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
reporterCodeWidget = widgets.Dropdown(
    options=[("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE ,
    # value=[("Angola",m49_angola)],
    description='Fonte (reporter):',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
partnerCodeWidget = widgets.Dropdown(
    options=[("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE ,
    # value=[("Angola",m49_angola)],
    description='Parceiro:',
    disabled=False,
)

# select flowCode widget from list M=Importações, X=Exportações
flowCodeWidget = widgets.Dropdown(
    options=[("Importações", "M"), ("Exportações", "X")],
    value="M",
    description='Fluxo:',
    disabled=False, 
)



### Escolher ano, parceiro e fluxo (import/export) para análise 

Pode escolher-se mais que um ano.

In [8]:

display(years)
display(reporterCodeWidget)
display(partnerCodeWidget)
display(flowCodeWidget)


SelectMultiple(description='Ano:', index=(17, 18), options=(2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 20…

Dropdown(description='Fonte (reporter):', index=8, options=(('China', 156), ('Macau', 446), ('Hong Kong', 344)…

Dropdown(description='Parceiro:', index=3, options=(('China', 156), ('Macau', 446), ('Hong Kong', 344), ('Ango…

Dropdown(description='Fluxo:', options=(('Importações', 'M'), ('Exportações', 'X')), value='M')

Depois de escolher ano, país e fluxo executar a célula seguinte.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [14]:
import comtrade
from comtrade import COUNTRY_CODES, HS_CODES, HS_CODES_DF, HS_CODES_L2_DF


yearsList = ",".join(list(map(str,years.value)))
partnerCode = partnerCodeWidget.value
reporterCode = reporterCodeWidget.value


# decode country
partner_name = comtrade.COUNTRY_CODES.get(partnerCode)
reporter_name = comtrade.COUNTRY_CODES.get(reporterCode)

flow = flowCodeWidget.value
print(reporter_name,yearsList,flow,partner_name,)

pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']

pco_cols_detail = ['reporterDesc','partnerDesc','refYear','cmdCodeAG2','motDesc','cmdCode','cmdDesc',
            'flowCode','primaryValue', 'isAggregate']

# first we collect the top commodity level 2 codes            
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     cmdCode="AG2",
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     period=yearsList,
                     timeout=30,
                     echo_url=True
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
# get the countries
countries = pco_top5.partnerDesc.unique()
country_cmd_top5_codes = dict()

# get the detailed commodity codes for the top of each country
for country in countries:
    l2_codes = pco_top5[pco_top5.partnerDesc == country]['cmdCode'].unique()
    print(country,l2_codes)
    hs_details = []
    for l2_code in l2_codes:
        l2_sub_codes = list(HS_CODES_DF[HS_CODES_DF.hscode.str.startswith(l2_code)]['hscode'])
        hs_details = hs_details + l2_sub_codes
    # print(hs_details)
    country_cmd_top5_codes[country] = hs_details.copy()

# now we fetch the detail
countryDesc = COUNTRY_CODES[partnerCode]
country_cmd_codes = ",".join(country_cmd_top5_codes[countryDesc])
print(country_cmd_codes)

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     #cmdCode=country_cmd_codes, # sometimes the detail commodity codes generates errors
                     cmdCode=None, # This gives all the commodities for the country and year 
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     partner2Code=0,
                     period=yearsList,
                     motCode=0,
                     echo_url=True,
                     timeout=60
                     )
                     
df['cmdCodeAG2'] = df.cmdCode.str[0:2]
# filter the detail commodity codes by the top 5
df = df[df['cmdCodeAG2'].isin(country_cmd_top5_codes[countryDesc])]



Portugal 2017,2018 M Angola
https://comtradeapi.un.org/public/v1/preview//C/A/HS?reporterCode=620&period=2017%2C2018&partnerCode=24&partner2Code=0&cmdCode=AG2&flowCode=M&customsCode=C00
Angola ['27' '84' '03']
27,2701,270111,270112,270119,270120,2702,270210,270220,2703,270300,2704,270400,2705,270500,2706,270600,2707,270710,270720,270730,270740,270750,270791,270799,2708,270810,270820,2709,270900,2710,271012,271019,271020,271091,271099,2711,271111,271112,271113,271114,271119,271121,271129,2712,271210,271220,271290,2713,271311,271312,271320,271390,2714,271410,271490,2715,271500,2716,271600,84,8401,840110,840120,840130,840140,8402,840211,840212,840219,840220,840290,8403,840310,840390,8404,840410,840420,840490,8405,840510,840590,8406,840610,840681,840682,840690,8407,840710,840721,840729,840731,840732,840733,840734,840790,8408,840810,840820,840890,8409,840910,840991,840999,8410,841011,841012,841013,841090,8411,841111,841112,841121,841122,841181,841182,841191,841199,8412,841210,841221,841229,

/Users/jrc/develop/cipf-comtrade/comtrade.py:309: UserWarning: Query returned different motCodes including 0 (all), check for duplicate trsults when aggregating. Use motCode = -1 to remove motCode = 0
  warnings.warn("Query returned different motCodes including 0 (all), check for duplicate trsults when aggregating. Use motCode = -1 to remove motCode = 0")


Temos que identificar as linhas com valor agregado porque
os dados não tem essa coluna correcta.

Na lista China-Angola importações a flag isAggregate parece não 
estar correcta até 2017 inclusive. 2018 já está bem.

See the [bug report](isaggregate_bug.ipynb)


In [10]:
import comtrade
# check which lines are aggregates
df.sort_values(['reporterDesc','partnerDesc','refYear','flowCode','motCode','cmdCode'], inplace=True)
print("check if isAggregate is correct before using the data")
print(df[['reporterDesc','partnerDesc','refYear','flowCode','cmdCodeAG2','motCode','cmdCode','isAggregate']].head(20))

df = comtrade.checkAggregateValues(df,'cmdCode','isAggregate2')
df.sort_values(['reporterDesc','partnerDesc','refYear','flowCode','cmdCode','motCode'], inplace=True)
df[['reporterDesc','partnerDesc','refYear','flowCode','cmdCodeAG2','cmdCode','motCode','isAggregate','isAggregate2']].head(20)



check if isAggregate is correct before using the data
    reporterDesc partnerDesc  refYear flowCode cmdCodeAG2  motCode cmdCode  \
0       Portugal      Angola     2017        M         03        0      03   
2       Portugal      Angola     2017        M         03        0    0303   
4       Portugal      Angola     2017        M         03        0  030342   
6       Portugal      Angola     2017        M         03        0  030357   
8       Portugal      Angola     2017        M         03        0  030381   
10      Portugal      Angola     2017        M         03        0    0306   
12      Portugal      Angola     2017        M         03        0  030614   
14      Portugal      Angola     2017        M         03        0  030617   
132     Portugal      Angola     2017        M         27        0      27   
135     Portugal      Angola     2017        M         27        0    2709   
137     Portugal      Angola     2017        M         27        0  270900   
139     Po

,reporterDesc,partnerDesc,refYear,flowCode,cmdCodeAG2,cmdCode,motCode,isAggregate,isAggregate2
0,Portugal,Angola,2017,M,03,03,0,True,True
1,Portugal,Angola,2017,M,03,03,2100,True,True
2,Portugal,Angola,2017,M,03,0303,0,True,True
3,Portugal,Angola,2017,M,03,0303,2100,True,True
4,Portugal,Angola,2017,M,03,030342,0,True,False
5,Portugal,Angola,2017,M,03,030342,2100,True,False
6,Portugal,Angola,2017,M,03,030357,0,True,False
7,Portugal,Angola,2017,M,03,030357,2100,True,False
8,Portugal,Angola,2017,M,03,030381,0,True,False
9,Portugal,Angola,2017,M,03,030381,2100,True,False


In [11]:

df['valueForTotal'] = df['primaryValue']
df.loc[df['isAggregate2'] == True,'valueForTotal'] = 0
df['subtotalAG2'] = df.groupby(['partnerCode','refYear','flowCode','cmdCodeAG2','motCode'])["valueForTotal"].transform('sum')
df['percentAG2'] = df['primaryValue'] / df['subtotalAG2'] * 100

In [12]:
import comtrade

excel_file_name = f"./reports/product_detail_{reporter_name}_{yearsList.replace(',','-')}_{flow}_{partner_name}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name, engine='xlsxwriter')

df2 = df[pco_cols_detail+['percentAG2']].sort_values('primaryValue',ascending=False)
df2.to_excel(excel_file, sheet_name='detail', index=True, startrow=2)

comtrade.excel_col_autowidth(df2, excel_file, 'detail')
comtrade.excel_format_currency(df2, excel_file,'detail', columns=['primaryValue'],width=20)
excel_file.close()
print(f"Excel file {excel_file_name} created")


Excel file ./reports/product_detail_Portugal_2017-2018_M_Angola.xlsx created


Existe um desdobramento de linhas por `motCode` (Mode of transport).
Aparece o total agregado que corresponde ao motCode=0 e os desdobramentos,
gerando valor repetido.

Um opção seria filtrar e só ter motCode=0. Contudo isso não pode
ser feito na query porque motCode não é um parametro de pesquisa.

Tem de ser feito depois dos dados obtidos.
As opções poderiam ser:
* motCode = None. Não fazer nada, talvez avisar se existir mais 
    que um motCode nos resultados e um for "0"
* motCode = 0. Só devolver linhas com motCode=0
* motCode = -1 Só devolver linhas com motCode != 0




In [13]:

pd.options.display.max_rows = 500
# df.sort_values(['partnerDesc','refYear','flowCode','subtotalAG2','cmdCodeAG2','primaryValue'],ascending=[True,True,True,False,True,False])[pco_cols_detail].head(500)
df.sort_values(['reporterDesc','partnerDesc','refYear','cmdCode','subtotalAG2'],ascending=[True,True,True,True,False])[pco_cols_detail+['isAggregate2','subtotalAG2','percentAG2','valueForTotal']].head(500)


,reporterDesc,partnerDesc,refYear,cmdCodeAG2,motDesc,cmdCode,cmdDesc,flowCode,primaryValue,isAggregate,isAggregate2,subtotalAG2,percentAG2,valueForTotal
0,Portugal,Angola,2017,03,All modes of transport,03,"Fish and crustaceans, molluscs and other aquatic invertebrates",M,"7,288,746.07",True,True,"7,288,746.07",100.00,0.00
1,Portugal,Angola,2017,03,Sea,03,"Fish and crustaceans, molluscs and other aquatic invertebrates",M,"7,288,746.07",True,True,"7,288,746.07",100.00,0.00
2,Portugal,Angola,2017,03,All modes of transport,0303,"Fish; frozen, excluding fish fillets and other fish meat of heading 0304",M,"219,615.40",True,True,"7,288,746.07",3.01,0.00
3,Portugal,Angola,2017,03,Sea,0303,"Fish; frozen, excluding fish fillets and other fish meat of heading 0304",M,"219,615.40",True,True,"7,288,746.07",3.01,0.00
4,Portugal,Angola,2017,03,All modes of transport,030342,"Fish; frozen, yellowfin tunas (Thunnus albacares), excluding fillets, fish meat of 0304, and edi...",M,"21,276.54",True,False,"7,288,746.07",0.29,"21,276.54"
5,Portugal,Angola,2017,03,Sea,030342,"Fish; frozen, yellowfin tunas (Thunnus albacares), excluding fillets, fish meat of 0304, and edi...",M,"21,276.54",True,False,"7,288,746.07",0.29,"21,276.54"
6,Portugal,Angola,2017,03,All modes of transport,030357,"Fish; frozen, swordfish (Xiphias gladius), excluding fillets, fish meat of 0304, and edible fish...",M,"28,331.06",True,False,"7,288,746.07",0.39,"28,331.06"
7,Portugal,Angola,2017,03,Sea,030357,"Fish; frozen, swordfish (Xiphias gladius), excluding fillets, fish meat of 0304, and edible fish...",M,"28,331.06",True,False,"7,288,746.07",0.39,"28,331.06"
8,Portugal,Angola,2017,03,All modes of transport,030381,"Fish; frozen, dogfish and other sharks, excluding fillets, fish meat of 0304, and edible fish of...",M,"170,007.80",True,False,"7,288,746.07",2.33,"170,007.80"
9,Portugal,Angola,2017,03,Sea,030381,"Fish; frozen, dogfish and other sharks, excluding fillets, fish meat of 0304, and edible fish of...",M,"170,007.80",True,False,"7,288,746.07",2.33,"170,007.80"
